In [101]:
import numpy as np
import random 
from tqdm.auto import tqdm
import heapq

## Generation of the random matrix of N dimension

In [102]:
#just a function to check if the randomly generated matrix is solvable or not
def is_solvable(puzzle, n):
    flattened = [tile for row in puzzle for tile in row if tile != 0]
    inversions = sum(1 for i in range(len(flattened)) for j in range(i+1, len(flattened)) if flattened[i]>flattened[j])
    if n%2 != 0:
        return inversions % 2 == 0
    else:
        blank_row = n -np.where(puzzle == 0)[0][0]
        return (blank_row % 2 == 0) != (inversions %2 == 0)                                  

In [103]:
def matrix_generator(n):
    while True:
        puzzle = np.random.permutation(n**2).reshape(n,n)
        if is_solvable(puzzle, n):
            return puzzle

## Implementaion of A*

In [104]:
#the helper function used to calculate the Manhattan Distance heuristic

def manhattan_distance(state, goal):
    n = state.shape[0] #it is the length of state as Dimension of the matrix
    distance = 0
    for i in range(n):
        for j in range(n):
            tile = state[i, j]
            if tile != 0:
                # goal_pos = divmod(goal.index(state[i,j]), n)
                goal_pos = np.argwhere(goal == tile)[0]
                distance += abs(i-goal_pos[0]) + abs(j-goal_pos[1])
        return distance

In [105]:
#generation of neighbouring states by moving the blank tilt

def get_neighbors(state):
    n = state.shape[0]
    neighbors = []
    x, y = np.argwhere(state == 0)[0]
    moves = [(-1, 0), (1,0), (0, -1), (0,1)] #top , bottom, Left, right
    for dx, dy in moves:
        nx, ny = x+dx, y+dy
        if 0<=nx <n and 0 <= ny <n:
            new_state = state.copy()
            new_state[x, y], new_state[nx, ny] = new_state[nx, ny], new_state[x,y]
            neighbors.append(new_state)
    return neighbors

In [ ]:
def a_star_search(start, goal):
    n = start.shape[0]
    goal_flat = [tile for row in goal for tile in row]  # Flatten goal for heuristic
    
    # Normalize to Python tuple of integers
    start_tuple = tuple(tuple(int(val) for val in row) for row in start)
    goal_tuple = tuple(tuple(int(val) for val in row) for row in goal)

    # Priority queue for A* (f(n), g(n), state, path)
    frontier = [(manhattan_distance(start, goal), 0, start_tuple, [])]
    heapq.heapify(frontier)
    visited = set()

    cost = 0  # Nodes evaluated

    while frontier:
        f, g, current, path = heapq.heappop(frontier)
        cost += 1

        # Convert `current` back to a NumPy array for processing
        current_state = np.array(current)

        # Debugging: Print the current state and frontier
        # print(f"Current State:\n{current_state}")
        # print(f"Frontier: {frontier}")

        # Check if the goal state is reached
        if np.array_equal(current_state, goal):
            return path, cost

        # Add the current state to visited (converted to tuple for hashability)
        visited.add(current)

        # Generate neighbors
        for neighbor in get_neighbors(current_state):
            # Normalize the neighbor to Python `int` tuples
            neighbor_tuple = tuple(tuple(int(val) for val in row) for row in neighbor)

            # Only add neighbor if not already visited
            if neighbor_tuple not in visited:
                h = manhattan_distance(neighbor, goal)

                # Push to the frontier, ensuring everything is tuples and integers
                heapq.heappush(frontier, (
                    g + 1 + h,
                    g + 1,
                    neighbor_tuple,
                    path + [neighbor_tuple]  # Use tuples here as well
                ))

    return None, cost  # No solution found


In [107]:
# Implentation of IDA*

def ida_star_search(start, goal):
    quality = 0
    cost = 0

    def search(state, g, threshold, path):
        nonlocal cost
        f = g + manhattan_distance(state, goal)
        
        # If the cost exceeds the threshold, return f-value
        if f > threshold:
            return f
        
        # If goal state is reached
        if np.array_equal(state, goal):
            quality = len(path)
            return path
        
        min_threshold = float('inf')
        
        # Explore neighbors
        for neighbor in get_neighbors(state):
            new_path = path + [neighbor]
            cost += 1
            temp_threshold = search(neighbor, g + 1, threshold, new_path)
            
            # If a solution is found, return the path
            if isinstance(temp_threshold, list):
                return temp_threshold
            
            # Otherwise, track the minimum threshold
            min_threshold = min(min_threshold, temp_threshold)
        
        return min_threshold
    
    # Iteratively deepen the threshold until the solution is found
    threshold = manhattan_distance(start, goal)
    while True:
        result = search(start, 0, threshold, [start])
        if isinstance(result, list):
            efficiency = quality / cost if cost != 0 else 0
            return result, quality, cost, efficiency  # Solution found
        threshold = result  # Increase the threshold

In [108]:
PUZZLE_DIM = 3
start = matrix_generator(PUZZLE_DIM)
goal = np.arange(1, PUZZLE_DIM**2).tolist() + [0]
goal = np.array(goal).reshape(PUZZLE_DIM,PUZZLE_DIM)

print("initial matrix is: ")
print(start)
print("the goal matrix is: ")
print(goal)

solution_path, total_cost = a_star_search(start, goal)
solution, quality, cost, efficiency = ida_star_search(start, goal)


print("\nSolution:")
for step in solution_path:
    print(np.array(step), "\n")
print("Output using A*")
print(f"Quality (Number of Moves): {len(solution_path)}") #length of the solution path in case of A* it is equivalent to the length of the solution path.
print(f"Cost (Nodes Evaluated): {total_cost}") #total no of state the algorithms has evaluated or processed in order to find the solution.
print(f"Efficiency (Quality / Cost): {len(solution_path) / total_cost:.4f}")

print()
print("output using IDA*")
print(f"Quality (Number of Moves): {quality}") #length of the solution path in case of A* it is equivalent to the length of the solution path.
print(f"Cost (Nodes Evaluated): {cost}") #total no of state the algorithms has evaluated or processed in order to find the solution.
print(f"Efficiency (Quality / Cost): {efficiency}")
# cost in A* algorithms is typically high because it is evaluating first all potential states before finding the goal state.
# Number of state grows rapidly with the size of matrix


initial matrix is: 
[[5 6 7]
 [0 2 3]
 [8 1 4]]
the goal matrix is: 
[[1 2 3]
 [4 5 6]
 [7 8 0]]


IndexError: index 1 is out of bounds for axis 0 with size 1